In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from pathlib import Path

sys.path.append("/home/acho/Sync/KiddLab/MSM/src")
from utils.stim_tools import *

sys.path.append("/home/acho/Sync/Python/sigtools")
from sigtools.representations import *
# from sigtools.sounds import *
# from sigtools.processing import *
# from sigtools.spatialization import *

|| MSM INITIALIZATION
Setting paths...
Importing sigtools...
Setting up sounddevice...
Importing level adjustments...
Setting eligible BUG words...
Set values for tone pattern synthesis...
Initialization complete!


In [13]:
def set_stim_order(stim_database, n_srcs, task_type, conditions,
                   n_blocks=None,
                   n_trials_per_block_per_condition=None,
                   n_trials_per_block=None,
                   n_blocks_per_condition=None,
                   randomize_within_block=True):
    if randomize_within_block:
        if n_trials_per_block or n_blocks_per_condition:
            raise ValueError
        n_trials_per_condition = n_blocks*n_trials_per_block_per_condition        
    else:
        if n_blocks or n_trials_per_block_per_condition:
            raise ValueError
        n_trials_per_condition = n_trials_per_block*n_blocks_per_condition
    
    temp = pd.DataFrame(columns=["cond", "stim_num", "pattern"])
    for cond in conditions:
        if cond == "co-located":
            cond_subset = \
                stim_database[(stim_database["n_srcs"] == n_srcs) &
                              (stim_database["stim_type"] == task_type) &
                              (stim_database["is_target"]) &
                              (stim_database["alt_rate"] == 0) &
                              (stim_database["init_angle"] == 0)].sample(n_trials_per_condition)
        elif cond == "plus_minus_90":
            cond_subset = \
                stim_database[(stim_database["n_srcs"] == n_srcs) &
                              (stim_database["stim_type"] == task_type) &
                              (stim_database["is_target"]) &
                              (stim_database["alt_rate"] == 0) &
                              (stim_database["init_angle"] != 0)].sample(n_trials_per_condition)
        else:
            cond_subset = \
                stim_database[(stim_database["n_srcs"] == n_srcs) &
                              (stim_database["stim_type"] == task_type) &
                              (stim_database["is_target"]) &
                              (stim_database["alt_rate"] == cond)].sample(n_trials_per_condition)
        cond_subset = cond_subset[["stim_num", "pattern"]]
        cond_subset.insert(0, "cond", len(cond_subset)*[str(cond)])
        temp = temp.append(cond_subset)
    
    run_stim_order = []
    if randomize_within_block:
        for block_num in range(n_blocks):
            curr_block = []
            for cond in conditions:
                curr_cond_df = temp[temp["cond"] == cond]
                sampled = curr_cond_df.sample(n_trials_per_block_per_condition, replace=False)
                temp = temp.drop(sampled.index)
                sampled_stim_nums = sampled["stim_num"].values.astype(int)
                sampled_stim_patterns = sampled["pattern"].values
                sampled_stim_patterns = [pattern.split(" ") for pattern in sampled_stim_patterns]
                sampled_stims = []
                for stim_num in sampled_stim_nums:
                    stim_path = STIM_DIR/("stim_" + str(stim_num).zfill(5) + ".wav")
                    stimulus = SoundLoader(stim_path)
                    sampled_stims.append(stimulus)
                curr_block += list(zip(sampled_stims, sampled_stim_nums, sampled_stim_patterns))
            np.random.shuffle(curr_block)
            run_stim_order.append(curr_block)
    else:
        for _ in range(n_blocks_per_condition):
            np.random.shuffle(conditions)
            for cond in conditions:
                curr_block = []
                curr_cond_df = temp[temp["cond"] == cond]
                sampled = curr_cond_df.sample(n_trials_per_block, replace=False)
                temp = temp.drop(sampled.index)
                sampled_stim_nums = sampled["stim_num"].values.astype(int)
                sampled_stim_patterns = sampled["pattern"].values
                sampled_stim_patterns = [pattern.split(" ") for pattern in sampled_stim_patterns]
                sampled_stims = []
                for stim_num in sampled_stim_nums:
                    stim_path = STIM_DIR/("stim_" + str(stim_num).zfill(5) + ".wav")
                    stimulus = SoundLoader(stim_path)
                    sampled_stims.append(stimulus)
                curr_block = list(zip(sampled_stims, sampled_stim_nums, sampled_stim_patterns))
                np.random.shuffle(curr_block)
                run_stim_order.append(curr_block)
    return run_stim_order

In [3]:
stim_database = pd.read_csv(STIM_DIR/"stimulus_database.csv")

In [ ]:
sss = set_stim_order(stim_database, 2, "SIM", ["co-located", "plus_minus_90"], 
                     n_blocks=2, n_trials_per_block_per_condition=3)

In [14]:
sss = set_stim_order(stim_database, 2, "SIM", ["co-located", "plus_minus_90"], 
                     n_trials_per_block=3, n_blocks_per_condition=1, randomize_within_block=False)

In [18]:
sss

[[(<sigtools.sounds.SoundLoader at 0x7f06244c32e8>,
   20,
   ['SUE', 'FOUND', 'SIX', 'CHEAP', 'SHOES']),
  (<sigtools.sounds.SoundLoader at 0x7f06244c5d68>,
   27,
   ['SUE', 'FOUND', 'THREE', 'NEW', 'BAGS']),
  (<sigtools.sounds.SoundLoader at 0x7f06244c50f0>,
   23,
   ['SUE', 'SOLD', 'THREE', 'BIG', 'SOCKS'])],
 [(<sigtools.sounds.SoundLoader at 0x7f06244c3780>,
   11,
   ['SUE', 'LOST', 'THREE', 'CHEAP', 'BAGS']),
  (<sigtools.sounds.SoundLoader at 0x7f06244c3748>,
   12,
   ['SUE', 'TOOK', 'SIX', 'HOT', 'SHOES']),
  (<sigtools.sounds.SoundLoader at 0x7f06244c37b8>,
   16,
   ['SUE', 'GAVE', 'FIVE', 'NEW', 'PENS'])]]

In [6]:
sss[0]

[(<sigtools.sounds.SoundLoader at 0x7f06244fb3c8>,
  21,
  ['SUE', 'FOUND', 'FIVE', 'BIG', 'HATS']),
 (<sigtools.sounds.SoundLoader at 0x7f06244fb320>,
  27,
  ['SUE', 'FOUND', 'THREE', 'NEW', 'BAGS']),
 (<sigtools.sounds.SoundLoader at 0x7f06244fb748>,
  13,
  ['SUE', 'SAW', 'EIGHT', 'CHEAP', 'SOCKS']),
 (<sigtools.sounds.SoundLoader at 0x7f06244fb358>,
  23,
  ['SUE', 'SOLD', 'THREE', 'BIG', 'SOCKS']),
 (<sigtools.sounds.SoundLoader at 0x7f06244fb710>,
  15,
  ['SUE', 'TOOK', 'SIX', 'GREEN', 'PENS']),
 (<sigtools.sounds.SoundLoader at 0x7f06244fb7b8>,
  17,
  ['SUE', 'SOLD', 'TEN', 'GREEN', 'PENS'])]

In [7]:
sss[1]

IndexError: list index out of range